In [1]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [9]:
image = cv2.imread('/home/amit/Shared/CTEVT/forms/person1/form.jpg')
 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, threshold1=100, threshold2=200)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


contours = sorted(contours, key=cv2.contourArea, reverse=True)

face = False

image_h, image_w = image.shape[:2]

print(image_h, image_w)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    gray_photo = gray[y:y+h, x:x+w]
    faces = face_cascade.detectMultiScale(gray_photo, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    print(len(faces))
    if len(faces)>=1 and (0.07*image_w < w < 0.2*image_w and 0.07*image_h < h < 0.2*image_h):
        passport_photo = image[y:y+h, x:x+w]
        passport_photo = cv2.resize(passport_photo, (192, 192),interpolation=cv2.INTER_LANCZOS4)
        face = True
        break

if not face:
    passport_photo = np.random.randint(0, 10, (192, 192))

cv2.imwrite('photo.jpg', passport_photo)

1376 1000
0
0
0
0
1


True

In [16]:
import os 
path = '.'
dirs = os.listdir(path, dir)

for dir in dirs:
    path = os.path.join(path, dir)
    print(path)

.\form.jpg
.\form.jpg\form2.jpg
.\form.jpg\form2.jpg\New folder
.\form.jpg\form2.jpg\New folder\New folder - Copy
.\form.jpg\form2.jpg\New folder\New folder - Copy\New folder - Copy (2)
.\form.jpg\form2.jpg\New folder\New folder - Copy\New folder - Copy (2)\New folder - Copy (3)
.\form.jpg\form2.jpg\New folder\New folder - Copy\New folder - Copy (2)\New folder - Copy (3)\photo.jpg
.\form.jpg\form2.jpg\New folder\New folder - Copy\New folder - Copy (2)\New folder - Copy (3)\photo.jpg\test.ipynb


In [35]:
def correct_rotation(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

    # Use Hough Line Transform to detect lines
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)

    # Initialize the angle
    angle = 0

    # If lines are found, compute the median angle of the lines
    if lines is not None:
        angles = []
        for rho, theta in lines[:, 0]:
            angle_in_degrees = np.degrees(theta) - 90  # Convert from radians to degrees
            angles.append(angle_in_degrees)
        
        # Use the median angle to correct the image rotation
        angle = np.median(angles)

    # Rotate the image to align it
    height, width = image.shape[:2]
    center = (width // 2, height // 2)

    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Apply the rotation matrix
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    return rotated_image

In [38]:
ph = cv2.imread('photo.jpg')
c = correct_rotation(ph)

In [39]:
cv2.imwrite('rotated.jpg', c)

True

### Correcting the rotation

In [1]:
import cv2
import dlib
import numpy as np

def align_face(image):

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector(gray)

    if len(faces) > 0:
        # Assuming the first detected face is the one we want to align
        face = faces[0]

        # Get facial landmarks
        landmarks = predictor(gray, face)

        # Get the coordinates of the eyes (use landmarks for the eyes)
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)

        # Compute the angle of rotation between the eyes
        delta_y = right_eye[1] - left_eye[1]
        delta_x = right_eye[0] - left_eye[0]
        angle = np.degrees(np.arctan2(delta_y, delta_x))  # Angle in degrees

        # Get the center of the image
        height, width = image.shape[:2]
        center = (width // 2, height // 2)

        # Rotation matrix to align the eyes horizontally
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Rotate the image
        rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

        return rotated_image
    else:
        print("No face detected!")
        return image

# Load the passport-size image
image = cv2.imread('rotated.jpg')

# Align the image (correct rotation based on face)
aligned_image = align_face(image)

# Show the result
cv2.imshow('Aligned Image', aligned_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the corrected image
cv2.imwrite('aligned_passport_image.jpg', aligned_image)


ModuleNotFoundError: No module named 'dlib'

In [21]:
import os

# Replace with the root directory you want to start the search from
root_dir = '.'

# Loop through all directories and files
for root, dirs, files in os.walk(root_dir):
    for file in files:
        p = os.path.join(root, file)
        if p.endswith('.jpg'):
            print(p)
        

.\form2.jpg
.\photo.jpg
.\New_folder\form.jpg
